In [8]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yaelsuarez (warm-kanelbullar). Use `wandb login --relogin` to force relogin


True

In [9]:
from diametery.line_fit import LineFit
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [13]:
with wandb.init(project="diameterY-simple", job_type="test", mode="online") as run:
    run.config.n_measurements = 10
    run.config.step_size = 0.3
    #Download dataset 2D fibers
    dataset_artifact = run.use_artifact("simple-fibers:v0")
    dataset_dir = dataset_artifact.download("dataset_files")
    model = LineFit(run.config.n_measurements, run.config.step_size)
    rows = []
    pbar = tqdm(desc="images")
    for f in os.listdir(dataset_dir): 
        if f.startswith("test") and f.endswith(".npz"):
            file_path = os.path.join(dataset_dir, f)
            example = np.load(file_path)
            diameter_pred = model.predict(example["x"])
            rows.append(dict(
                image=wandb.Image(example["x"], caption=os.path.splitext(f)[0]), 
                d=example["d"],
                d_pred=diameter_pred
            ))
            pbar.update()
    df = pd.DataFrame(rows)
    print(df)

wandb: Downloading large artifact simple-fibers:v0, 802.80MB. 3842 files... Done. 0:0:0.5
images: 0it [00:34, ?it/s]
images: 25it [00:00, 33.97it/s]/Users/carmenlopez/dev/diameterY/diametery/line_fit.py:113: RuntimeWarning: Mean of empty slice.
  calculated_diameter = np.array(diameters).mean()
/Users/carmenlopez/miniconda3/envs/diametery/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
images: 57it [00:01, 32.60it/s]/Users/carmenlopez/miniconda3/envs/diametery/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
images: 127it [00:04, 22.61it/s]

                                                 image   d     d_pred
0    <wandb.sdk.data_types.image.Image object at 0x...  27  27.000000
1    <wandb.sdk.data_types.image.Image object at 0x...  34  33.111111
2    <wandb.sdk.data_types.image.Image object at 0x...  34  33.777778
3    <wandb.sdk.data_types.image.Image object at 0x...  31  31.000000
4    <wandb.sdk.data_types.image.Image object at 0x...  14  13.222222
..                                                 ...  ..        ...
123  <wandb.sdk.data_types.image.Image object at 0x...  55  56.000000
124  <wandb.sdk.data_types.image.Image object at 0x...  58  57.500000
125  <wandb.sdk.data_types.image.Image object at 0x...  60  59.333333
126  <wandb.sdk.data_types.image.Image object at 0x...  33  32.571429
127  <wandb.sdk.data_types.image.Image object at 0x...  51  51.833333

[128 rows x 3 columns]


In [20]:

#(a-r)/r
df["Error_abs"] =  abs((df["d_pred"] - df["d"])/ df["d"])
abs_error = df["Error_abs"].mean()*100
print(abs_error)

1.7641572016223948
